# Preamble

In [1]:
import xarray as xr
import numpy as np
import holoviews as hv
from pathlib import Path
import pandas as pd
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import numpy as np
from datetime import date
from matplotlib import pyplot as plt
# matplotlib widget
# import marineHeatWaves as mhw
# from marineHeatWaves import detect
import dask.array as da
# from dask.distributed import Client, LocalCluster
# # import param
# from holoviews import streams
import csv
# Get working directory
import sys, os
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Import Air data

In [2]:
# Air temperature serie
air_df = pd.read_csv('../../data/Excel/Bardolino_air_temp_predict.csv', usecols=[0, 1], parse_dates=[0])
air_df.columns = ['Date', 'Air_Temp']
# Define the date range
start_date = '1992-01-01'
end_date = '2022-12-31'

# Filter the DataFrame to retain only the rows within the specified date range
air_df = air_df[(air_df['Date'] >= start_date) & (air_df['Date'] <= end_date)]
air_df.reset_index(drop=True, inplace=True)
air_df

,Date,Air_Temp
0,1992-01-01,2.433684
1,1992-01-02,-2.230208
2,1992-01-03,-0.533333
3,1992-01-04,0.846875
4,1992-01-05,0.875000
...,...,...
11318,2022-12-27,7.542708
11319,2022-12-28,7.419792
11320,2022-12-29,6.354167
11321,2022-12-30,6.654167


## Import Satellite Original exported data

In [3]:
# SET PARAMETERS

# Buffer: filter out cells close to the land [km]
buffer = 0.8

# Quality: minimum value of satellite data quality to consider
quality_thresh = 5 


In [4]:
# linear interpolated datasets
ex_fn = 'extracted/ID505-lago_di_garda-LSWT_LSWTflag_LIC-19920926_20221231-v2.1.0.extracted.nc'

# set full paths
ex_path_lswt = Path(module_path).joinpath('../data/'+ex_fn)

# load lswt dataset as xarray.Dataset
ds_ex = xr.open_dataset(ex_path_lswt, chunks='auto')
ds_ex

<xarray.Dataset>
Dimensions:                         (time: 11054, lat: 52, lon: 43)
Coordinates:
  * time                            (time) datetime64[ns] 1992-09-26 ... 2022...
  * lat                             (lat) float64 45.45 45.45 ... 45.86 45.87
  * lon                             (lon) float64 10.51 10.52 ... 10.85 10.86
Data variables:
    lake_ice_cover_class            (time, lat, lon) float32 dask.array<chunksize=(11054, 52, 43), meta=np.ndarray>
    lake_surface_water_temperature  (time, lat, lon) float32 dask.array<chunksize=(11054, 52, 43), meta=np.ndarray>
    lswt_quality_level              (time, lat, lon) float32 dask.array<chunksize=(11054, 52, 43), meta=np.ndarray>
    lakemask                        (lat, lon) float32 dask.array<chunksize=(52, 43), meta=np.ndarray>
    distance_to_land                (lat, lon) float32 dask.array<chunksize=(52, 43), meta=np.ndarray>
Attributes: (12/45)
    title:                      ESA Lakes_cci product
    institution:                LWL: Laboratoire d'Etudes en Geophysique et O...
    source:                     LWL: European Space Agency (ESA), National Ae...
    history:                    LWL: Generated by Laboratoire d'Etudes en Geo...
    references:                 https://climate.esa.int/en/projects/lakes/
    tracking_id:                af8cefca-75c7-4b43-b316-a7ff7a870c0d
    ...                         ...
    geospatial_lat_resolution:  0.008333333
    geospatial_lon_resolution:  0.008333333
    doi:                        10.5285/7fc9df8070d34cacab8092e45ef276f1
    lakename:                   lago_di_garda
    lakeid:                     505
    lakecells:                  613

In [5]:
# access lswt data
da_ex_lswt = ds_ex.lake_surface_water_temperature
# Convert from Kelvin to Celcius
da_ex_lswt = da_ex_lswt - 273.15
da_ex_lswt.attrs['description'] = 'Lake Surface Water Temperature'
da_ex_lswt.attrs['unit'] = 'degrees Celsius (°C)'
da_ex_lswt

<xarray.DataArray 'lake_surface_water_temperature' (time: 11054, lat: 52,
                                                    lon: 43)>
dask.array<sub, shape=(11054, 52, 43), dtype=float32, chunksize=(11054, 52, 43), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-26 1992-09-27 ... 2022-12-31
  * lat      (lat) float64 45.45 45.45 45.46 45.47 ... 45.85 45.85 45.86 45.87
  * lon      (lon) float64 10.51 10.52 10.53 10.54 ... 10.84 10.85 10.85 10.86
Attributes:
    description:  Lake Surface Water Temperature
    unit:         degrees Celsius (°C)

In [6]:
da_ex_quality = ds_ex.lswt_quality_level
# Filter to keep only quality >= threhsold
da_ex_lswt_filtered_quality = da_ex_lswt.where(da_ex_quality >= quality_thresh)

In [7]:
# Set the buffer
da_ex_lswt_filtered_quality_buffer = da_ex_lswt_filtered_quality.where(ds_ex['distance_to_land']>= buffer)

In [8]:
# Filter out data with lswt <4°C
da_ex_lswt_filtered_quality_buffer_4 = da_ex_lswt_filtered_quality_buffer.where(da_ex_lswt_filtered_quality_buffer >= 4)

In [9]:
da_lswt = da_ex_lswt_filtered_quality_buffer_4

# Satellite data series export

In [10]:
da_lswt

<xarray.DataArray 'lake_surface_water_temperature' (time: 11054, lat: 52,
                                                    lon: 43)>
dask.array<where, shape=(11054, 52, 43), dtype=float32, chunksize=(11054, 52, 43), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-26 1992-09-27 ... 2022-12-31
  * lat      (lat) float64 45.45 45.45 45.46 45.47 ... 45.85 45.85 45.86 45.87
  * lon      (lon) float64 10.51 10.52 10.53 10.54 ... 10.84 10.85 10.85 10.86
Attributes:
    description:  Lake Surface Water Temperature
    unit:         degrees Celsius (°C)

In [11]:
# Count number of cells to analyze
# Count non-null values along the 'time' dimension
count_temp = da_lswt.count(dim='time')

# Find the cells that have temperature data at least once
cells_with_data = count_temp > 0

# Sum over the 'lat' and 'lon' dimensions
total_cells_with_data = cells_with_data.sum(dim=['lat', 'lon'])

print("Total number of cells containing temperature data over time:", int(total_cells_with_data))

Total number of cells containing temperature data over time: 462


In [12]:
plot = count_temp.hvplot(
    geo=True,
    xlabel='Longitude', ylabel='Latitude',
    clabel='LSWT (°C)',
    tiles='CartoLight',
    width=300,
)
plot

:Overlay
   .Tiles.I :Tiles   [x,y]
   .Image.I :Image   [lon,lat]   (lake_surface_water_temperature)

In [13]:
# Extract the lat and lon indices where data are present
lat_lon_pairs = np.column_stack(np.where(cells_with_data))

In [14]:
def calculate_split_date(dataset):
 
    # Drop rows where the second column is NaN and reset the index
    non_nan_dataset = dataset.dropna(subset=[dataset.columns[1]]).reset_index(drop=True)
    
    # Calculate the position at 1/3 of all non-NaN values
    total_non_nan = len(non_nan_dataset)
    position = total_non_nan // 3
    
    # Get the date at the calculated position
    split_date = non_nan_dataset.iloc[position]['Date']
    
    # Identify the year and round to January 1 of the appropriate year
    split_year = split_date.year
    if split_date.month > 6 or (split_date.month == 6 and split_date.day > 15):
        split_year += 1
    
    split_date_january_first = pd.Timestamp(year=split_year, month=1, day=1)
    
    return split_date_january_first

In [15]:
# Initialize a dictionary to hold the DataFrames
dataframes = {}

# Loop through each lat and lon pair with data
for lat_idx, lon_idx in lat_lon_pairs:
    lat = da_lswt.lat.values[lat_idx]
    lon = da_lswt.lon.values[lon_idx]
    
    # Extract the time series for the current cell
    cell_data = da_lswt.isel(lat=lat_idx, lon=lon_idx)
    
    # Create a DataFrame with Date and Lswt
    df = pd.DataFrame({
        'Date': cell_data['time'].values,
        'Lswt': cell_data.values
    })
    
    # Calculate the split date
    split_date_january_first = calculate_split_date(df)
    
    # Extract the year from the split date
    split_year = split_date_january_first.year
    
    # Assign the DataFrame to the dictionary with a name based on lat, lon, and split year
    dataframes[f"{lat_idx}_{lon_idx}_{split_year}"] = df

In [16]:
# Check the dictionary to see the DataFrames created
for name, df in dataframes.items():
    print(f"{name}:\n{df.head()}\n")

1_21_2011:
        Date  Lswt
0 1992-09-26   NaN
1 1992-09-27   NaN
2 1992-09-28   NaN
3 1992-09-29   NaN
4 1992-09-30   NaN

2_17_2010:
        Date  Lswt
0 1992-09-26   NaN
1 1992-09-27   NaN
2 1992-09-28   NaN
3 1992-09-29   NaN
4 1992-09-30   NaN

2_18_2010:
        Date  Lswt
0 1992-09-26   NaN
1 1992-09-27   NaN
2 1992-09-28   NaN
3 1992-09-29   NaN
4 1992-09-30   NaN

2_19_2010:
        Date  Lswt
0 1992-09-26   NaN
1 1992-09-27   NaN
2 1992-09-28   NaN
3 1992-09-29   NaN
4 1992-09-30   NaN

2_20_2010:
        Date  Lswt
0 1992-09-26   NaN
1 1992-09-27   NaN
2 1992-09-28   NaN
3 1992-09-29   NaN
4 1992-09-30   NaN

2_21_2010:
        Date  Lswt
0 1992-09-26   NaN
1 1992-09-27   NaN
2 1992-09-28   NaN
3 1992-09-29   NaN
4 1992-09-30   NaN

2_22_2010:
        Date  Lswt
0 1992-09-26   NaN
1 1992-09-27   NaN
2 1992-09-28   NaN
3 1992-09-29   NaN
4 1992-09-30   NaN

3_6_2010:
        Date  Lswt
0 1992-09-26   NaN
1 1992-09-27   NaN
2 1992-09-28   NaN
3 1992-09-29   NaN
4 1992-09-30 

In [17]:
# Iterate over each DataFrame in the dictionary
for name, df in dataframes.items():
    # Merge with air_df
    df_merged = pd.merge(air_df, df, on='Date', how='left')
    
    # Replace NaN values in the 'Lswt' column with -999
    df_merged['Lswt'].fillna(-999, inplace=True)
    
    # Format numeric columns with specific decimal places
    df_merged['Air_Temp'] = df_merged['Air_Temp'].map('{:.3f}'.format)
    df_merged['Lswt'] = df_merged['Lswt'].map('{:.3f}'.format)
    
    # Update the dictionary with the merged and formatted DataFrame
    dataframes[name] = df_merged


In [18]:
# Check the dictionary to see the merged and formatted DataFrames
for name, df in dataframes.items():
    print(f"{name}:\n{df.head()}\n")

1_21_2011:
        Date Air_Temp      Lswt
0 1992-01-01    2.434  -999.000
1 1992-01-02   -2.230  -999.000
2 1992-01-03   -0.533  -999.000
3 1992-01-04    0.847  -999.000
4 1992-01-05    0.875  -999.000

2_17_2010:
        Date Air_Temp      Lswt
0 1992-01-01    2.434  -999.000
1 1992-01-02   -2.230  -999.000
2 1992-01-03   -0.533  -999.000
3 1992-01-04    0.847  -999.000
4 1992-01-05    0.875  -999.000

2_18_2010:
        Date Air_Temp      Lswt
0 1992-01-01    2.434  -999.000
1 1992-01-02   -2.230  -999.000
2 1992-01-03   -0.533  -999.000
3 1992-01-04    0.847  -999.000
4 1992-01-05    0.875  -999.000

2_19_2010:
        Date Air_Temp      Lswt
0 1992-01-01    2.434  -999.000
1 1992-01-02   -2.230  -999.000
2 1992-01-03   -0.533  -999.000
3 1992-01-04    0.847  -999.000
4 1992-01-05    0.875  -999.000

2_20_2010:
        Date Air_Temp      Lswt
0 1992-01-01    2.434  -999.000
1 1992-01-02   -2.230  -999.000
2 1992-01-03   -0.533  -999.000
3 1992-01-04    0.847  -999.000
4 1992-01-05 

In [19]:
# Split into calibration and validation
# Initialize dictionaries to hold the split DataFrames
dataframes_cc = {}
dataframes_cv = {}

# Iterate over each DataFrame in the dictionary
for name, df in dataframes.items():
    # Extract the year from the DataFrame's name
    year = int(name.split('_')[-1])
    split_date = pd.Timestamp(year=year, month=1, day=1)
    
    # Split the DataFrame based on the split date
    df_cc = df[df['Date'] >= split_date].copy()
    df_cv = df[df['Date'] < split_date].copy()
    
    # Generate new names for the split DataFrames
    name_cc = name.replace(f'_{year}', '_cc')
    name_cv = name.replace(f'_{year}', '_cv')
    
    # Store the split DataFrames in the new dictionaries
    dataframes_cc[name_cc] = df_cc
    dataframes_cv[name_cv] = df_cv

# Check the dictionaries to see the split DataFrames
for name, df in dataframes_cc.items():
    print(f"{name}:\n{df.head()}\n")

for name, df in dataframes_cv.items():
    print(f"{name}:\n{df.head()}\n")

1_21_cc:
           Date Air_Temp      Lswt
6940 2011-01-01    1.809  -999.000
6941 2011-01-02    2.034  -999.000
6942 2011-01-03    2.415  -999.000
6943 2011-01-04    0.106  -999.000
6944 2011-01-05   -0.857  -999.000

2_17_cc:
           Date Air_Temp      Lswt
6575 2010-01-01    4.806  -999.000
6576 2010-01-02    5.004  -999.000
6577 2010-01-03    1.858  -999.000
6578 2010-01-04   -0.569  -999.000
6579 2010-01-05    1.300  -999.000

2_18_cc:
           Date Air_Temp      Lswt
6575 2010-01-01    4.806  -999.000
6576 2010-01-02    5.004  -999.000
6577 2010-01-03    1.858  -999.000
6578 2010-01-04   -0.569  -999.000
6579 2010-01-05    1.300  -999.000

2_19_cc:
           Date Air_Temp      Lswt
6575 2010-01-01    4.806  -999.000
6576 2010-01-02    5.004  -999.000
6577 2010-01-03    1.858  -999.000
6578 2010-01-04   -0.569  -999.000
6579 2010-01-05    1.300  -999.000

2_20_cc:
           Date Air_Temp      Lswt
6575 2010-01-01    4.806  -999.000
6576 2010-01-02    5.004  -999.000
6577 2

In [20]:
# Define the function to format and export DataFrames
def format_and_export(dataframes, suffix):
    for name, df in dataframes.items():
        # Add Year, Month, and Day columns
        df['Year'] = pd.to_datetime(df['Date']).dt.year
        df['Month'] = pd.to_datetime(df['Date']).dt.month
        df['Day'] = pd.to_datetime(df['Date']).dt.day
        
        # Drop the Date column
        df.drop(columns=['Date'], inplace=True)
        
        # Reorder columns
        df = df[['Year', 'Month', 'Day', 'Air_Temp', 'Lswt']]
        
        # Define the file path
        file_path = f"../air2water-master/Garda/{name}.txt"
        
        # Export the DataFrame to a .txt file
        df.to_csv(file_path, sep='\t', index=False, header=False)
        
# Format and export the calibration DataFrames
format_and_export(dataframes_cc, '_cc')

# Format and export the validation DataFrames
format_and_export(dataframes_cv, '_cv')


# Model automation

In [21]:
# Define the path for the coordinates.txt file
coordinates_file_path = "../air2water-master/Garda/coordinates.txt"

# Open the coordinates.txt file and write the coordinates
with open(coordinates_file_path, "w") as coordinates_file:
    for lat_idx, lon_idx in lat_lon_pairs:
        lat = da_lswt.lat.values[lat_idx]
        lon = da_lswt.lon.values[lon_idx]
        coordinates_file.write(f"{lat_idx}_{lon_idx}\n")

In [22]:
for lat_idx, lon_idx in lat_lon_pairs:
    lat = da_lswt.lat.values[lat_idx]
    lon = da_lswt.lon.values[lon_idx]
    print(f"{lat_idx}_{lon_idx}")

1_21
2_17
2_18
2_19
2_20
2_21
2_22
3_6
3_7
3_8
3_13
3_14
3_15
3_16
3_17
3_18
3_19
3_20
3_21
3_22
3_23
4_5
4_6
4_7
4_8
4_9
4_10
4_13
4_14
4_15
4_16
4_17
4_18
4_19
4_20
4_21
4_22
4_23
5_5
5_6
5_7
5_8
5_9
5_10
5_13
5_14
5_15
5_16
5_17
5_18
5_19
5_20
5_21
5_22
5_23
5_24
6_2
6_3
6_4
6_5
6_6
6_7
6_8
6_9
6_13
6_14
6_15
6_16
6_17
6_18
6_19
6_20
6_21
6_22
6_23
6_24
6_25
7_2
7_3
7_4
7_5
7_6
7_7
7_8
7_9
7_10
7_12
7_13
7_14
7_15
7_16
7_17
7_18
7_19
7_20
7_21
7_22
7_23
7_24
7_25
8_3
8_4
8_5
8_6
8_7
8_8
8_9
8_10
8_11
8_12
8_13
8_14
8_15
8_16
8_17
8_18
8_19
8_20
8_21
8_22
8_23
8_24
8_25
9_5
9_6
9_7
9_8
9_9
9_10
9_11
9_12
9_13
9_14
9_15
9_16
9_17
9_18
9_19
9_20
9_21
9_22
9_23
9_24
10_7
10_8
10_9
10_10
10_11
10_12
10_13
10_14
10_15
10_16
10_17
10_18
10_19
10_20
10_21
10_22
10_23
10_24
11_7
11_8
11_9
11_10
11_11
11_12
11_13
11_14
11_15
11_16
11_17
11_18
11_19
11_20
11_21
11_22
11_23
11_24
12_8
12_9
12_10
12_11
12_12
12_13
12_14
12_15
12_16
12_17
12_18
12_19
12_20
12_21
12_22
12_23
13_9
13_10
13_11
13_12

In [23]:
# Now run the PowerShell script